In [1]:
import numpy as np
import math

In [2]:
#These points are computed in a separate file
Points=[(0, 0, 0), (0, 3, -2), (1, -2, 1), (1, 1, -1), (-2, 0, 1), (2, -1, 0), (-1, -2, 2), (-1, 1, 0), (3, 0, -1), (0, -1, 1), (0, 2, -1), (1, -3, 2), (1, 0, 0), (-2, -1, 2), (-2, 2, 0), (2, 1, -1), (-1, 0, 1), (-1, 3, -1), (0, -2, 2), (0, 1, 0), (-3, 0, 2), (1, -1, 1), (1, 2, -1), (-2, 1, 1), (2, 0, 0), (-1, -1, 2), (-1, 2, 0), (3, 1, -1), (0, 0, 1), (0, 3, -1), (1, -2, 2), (1, 1, 0), (-2, 0, 2), (2, -1, 1), (2, 2, -1), (-1, 1, 1), (3, 0, 0), (0, -1, 2), (0, 2, 0), (1, -3, 3), (1, 0, 1), (1, 3, -1), (-2, 2, 1), (2, 1, 0), (-1, 0, 2), (-1, 3, 0), (0, -2, 3), (0, 1, 1), (0, 4, -1), (1, -1, 2), (1, 2, 0), (-2, 1, 2), (2, 0, 1), (-1, -1, 3), (-1, 2, 1), (3, 1, 0), (0, 0, 2), (0, 3, 0), (1, -2, 3), (1, 1, 1), (-2, 0, 3), (2, -1, 2), (2, 2, 0), (-1, 1, 2), (3, 0, 1), (0, -1, 3), (0, 2, 1), (4, 1, 0), (1, 0, 2), (1, 3, 0), (-2, 2, 2), (2, 1, 1), (-1, 0, 3), (-1, 3, 1), (3, 2, 0), (0, 1, 2), (0, 4, 0), (1, -1, 3), (1, 2, 1), (-2, 1, 3), (2, 0, 2), (2, 3, 0), (-1, 2, 2), (3, 1, 1), (0, 0, 3), (0, 3, 1), (1, -2, 4), (1, 1, 2), (1, 4, 0), (2, -1, 3), (2, 2, 1), (-1, 1, 3), (3, 0, 2), (0, -1, 4), (0, 2, 2), (4, 1, 1), (1, 0, 3), (1, 3, 1), (-2, 2, 3), (2, 1, 2), (-1, 0, 4), (-1, 3, 2), (3, 2, 1), (0, 1, 3), (0, 4, 1), (1, -1, 4), (1, 2, 2), (1, 5, 0), (2, 0, 3), (2, 3, 1), (-1, 2, 3), (3, 1, 2), (0, 0, 4), (0, 3, 2), (4, 2, 1), (1, 1, 3), (1, 4, 1), (2, -1, 4), (2, 2, 2), (-1, 1, 4), (3, 0, 3), (3, 3, 1), (0, 2, 3), (4, 1, 2), (1, 0, 4), (1, 3, 2), (-2, 2, 4), (2, 1, 3)]

In [3]:
#tones n range from 0 to 127

# This function transforms pitch to the representative of a lattice point
def PointFromTone(n):
    return np.array(Points[n])

# This function transforms lattice point to the corresponding pitch
def ToneFromPoint(A):
    return round(12*A[0]+19*A[1]+28*A[2])

#check the functions are inverses of each other 
#for n in range(0,128):
#    print(ToneFromPoint(PointFromTone(n)))

In [4]:
# Points array contains a single representative for each perceived pitch
# manyPoints array contains several representatives for each pitch

manyPoints=[]
v1=np.array([3,4,-4])
v2=np.array([-4,4,-1])
for n in range(0,128):
    for k in range(-2,2):
        for l in range(-2,2):
            manyPoints.append(np.array(Points[n])+k*v1+l*v2)

# In the following: layer = area between two parallel planes in R^3.
# Skew layer = layer 0<12x+19y+28z<128 
# Perceived range of sounds = integral points inside the skew layer

# The next function takes a vector as an input. If the vector is outside 
# the skew layer, the function outputs the closest lattice point inside the range
            
def PushToRange(v):
    sound=ToneFromPoint(v)
    if sound>=0 and sound<=127 :
        return v
    else :
#this returns the closest point among those perceived by human ear
        min=10000
        argmin=[0,0,0]
        for w in manyPoints :
            dist=np.linalg.norm(np.array(w)-v)
            if dist<min :
                argmin=np.array(w)
                min=dist
        return argmin

In [5]:
# we shift the origin to the point which is "in the middle" of the perceived range. 
# We assume it corresponds to the median of all tones

center = PointFromTone(63)
#print(center)

# f is (any) function on R^3
# Transformator allows to build a map of perceived 3d-lattices 
# from a given map f of 3d-space to itself 

def Transformator(f,n):
    oldPoint=PointFromTone(n)-center
    newPoint=f(oldPoint)+center
    newPointRounded=(round(newPoint[0]),round(newPoint[1]),round(newPoint[2]))
    perceptedPoint=PushToRange(newPointRounded)
    newTone=ToneFromPoint(perceptedPoint)
    return newTone    

In [6]:
# The next lines allow to construct maps of 3d-space, 
# which preserve the perceived range of sounds

# To simplify the task, we first map the skew layer to the horizontal layer, 
# then construct a diffeomorphism of the horizontal layer, 
# then go back to the skew layer 

# To make the map of horizontal layer, we perform lattice automorphism horizontally, and some
# homeomorphism in the vertical direction

# The transformation of the perceived skew layer to the horizontal layer
# is done by a fixed invertible linear transformation
Change = np.array([[3,-4,12],[4,4,19],[-4,-1,28]])
Inverse=np.linalg.inv(Change)

#This is the width of our tonal layer. We need it to make homeomorphism in the normal direction
Hwidth=127/np.linalg.norm(np.array([12,19,28]))/2
#print(width)

# This function takes as input an element of SL(2,Z) 
# (which is supposed to preserve the integral lattice)
# a shift vector (for an affine transformation of a torus)
# and a homeomorphism of an interval [-w/2,w/2]
# It outputs a homeomorphism of a tonal space (considered as a skew layer)

def FunctionGenerator(B,homeo,shift) :
    def F(arg) :
        flattened=Inverse.dot(arg)
        xy=np.array([flattened[0],flattened[1]])
        xy=np.array(B).dot(xy)
        z=homeo(flattened[2])
        new=np.array([xy[0]+shift[0]*z,xy[1]+shift[1]*z,z])
        return Change.dot(new)
    return F

# This is an example of the input data. These data can be changed. 
# LAut can be changed to any integer matrix with determinant +1 or -1.
# shift is any 2-vector  
LAut=[[1,1],[0,1]]
shift=[0.1,-0.15]

def homeo(x) :
    #return (math.sin((x/Hwidth)*math.pi/2))*Hwidth
    return x

F=FunctionGenerator(LAut,homeo,shift)

#print(F([0,1,2]))


# Simple substitution model:
# A pitch is substituted with another pitch, 
# computed according to the given map of the tonal space.

# mapping is an array which stores, for each pitch i, a pitch which substitutes i

Mapping = []
for n in range(0,128):
    Mapping.append(Transformator(F,n))

print(Mapping)

[9, 1, 8, 0, 4, 8, 3, 7, 11, 6, 10, 14, 18, 13, 17, 21, 16, 20, 24, 28, 23, 27, 19, 14, 18, 22, 26, 21, 25, 29, 24, 28, 32, 36, 31, 35, 39, 34, 38, 42, 46, 41, 45, 49, 44, 48, 52, 56, 51, 55, 47, 42, 46, 50, 54, 49, 53, 57, 52, 56, 60, 64, 59, 63, 67, 62, 66, 70, 74, 69, 73, 77, 72, 76, 80, 84, 79, 71, 75, 70, 74, 78, 82, 77, 81, 85, 80, 84, 88, 92, 87, 91, 95, 90, 94, 98, 102, 97, 101, 105, 100, 104, 108, 112, 107, 99, 103, 98, 102, 106, 110, 105, 109, 113, 108, 112, 116, 120, 115, 119, 123, 118, 122, 126, 102, 125, 101, 105]


In [7]:
# The next model is more involved "quantum type". 
# We substitute each pitch with a point of the skew layer. 
# The image point lies between several integer points representing pitches.
# We take all these pitches into account with weights corresponding to barycentric
# coordinates of the image points. We first need to subdivide the cubical lattice into 
# tetrahedra to make use of barycentric coordinates. 
# Triangulation is done empirically.

# In the end we get a 128x128 transformation matrix. 
# Application of this matrix to the vector of amplitudes (or durations) of pitches 
# gives a new vector of amplitudes (or durations).

#This computes baryc. coordinates of w relative to v1,v2,v3,v4. All vectors are 3-dim.
def BarycCoords(w,ListOf4Vecs):
    Matr=[]
    for v in ListOf4Vecs:
        Matr.append([v[0],v[1],v[2],1])
    Matr=np.array(Matr)
    w=np.array([w[0],w[1],w[2],1])
    x=w.dot(np.linalg.inv(Matr))
    return x

#print(BarycCoords([1,1,1],[[0,0,0],[3,0,0],[0,3,0],[0,0,3]]))

#This function checks in which simplex of [0,1]^3 lies a point 
#and outputs the list of its vertices

def CheckSimplex(vec):
    a,b,c=vec
    e0=np.array([0,0,0])
    e1=np.array([1,0,0])
    e2=np.array([0,1,0])
    e3=np.array([0,0,1])
    if a+b+c<=1 :
        return [e0,e1,e2,e3]
    elif a+b+c>=2 :
        return [e1+e2,e1+e3,e2+e3,e1+e2+e3]
    elif a+c>=1 and b+c>=1 :
        return [e3,e1+e2,e1+e3,e2+e3]
    elif a+c>=1 and b+c<=1 :
        return [e1,e3,e1+e3,e1+e2]
    elif a+c<=1 and b+c>=1 :
        return [e2,e3,e1+e2,e2+e3]
    elif a+c<=1 and b+c<=1 :
        return [e1,e2,e1+e2,e3]
    else:
        return "Fail"

# This function transforms each pitch without rounding
# and outputs a vector of weights. i-th component of the output is the weight of i-th pitch
# after transformation

def SmudgeTransformator(f,n):
    oldPoint=PointFromTone(n)-center
    newPoint=f(oldPoint)+center
    intPart=np.array([int(newPoint[0]//1),int(newPoint[1]//1),int(newPoint[2]//1)])
    fracPart=np.array([newPoint[0]%1,newPoint[1]%1,newPoint[2]%1])
    Vecs = CheckSimplex(fracPart)
    coords=BarycCoords(fracPart,Vecs)
    ans = [0] * 128
    for i in range(4):
        perceptedPoint=PushToRange(intPart+Vecs[i])
        curPitch=ToneFromPoint(perceptedPoint)
        ans[curPitch]+=coords[i]
        #print(curPitch)
    return ans

TransMatrix=[]
for i in range(128):
    TransMatrix.append(SmudgeTransformator(F,i))

In [8]:
# We store the transformation matrix in a separate file for future use

with open('TransitionMatrix.npy', 'wb') as f:
    np.save(f, TransMatrix)